In [176]:
import pandas as pd
import numpy as np

# replace 'your_file.csv' with your actual file path
df = pd.read_csv('savant_data.csv')

column_names = df.columns.values.tolist()
unique_values = df['pitch_name'].unique()
print(unique_values)
# Print the first 5 rows of the DataFrame
df = df.drop(columns=['zone','release_speed','game_pk','release_pos_x','release_pos_z','spin_dir','spin_rate_deprecated','break_angle_deprecated','break_length_deprecated','p_throws','home_team','away_team','description','hit_location','bb_type'])
df = df.drop(columns=['pitcher','game_year','pfx_x','pfx_z','plate_x','plate_z','hc_x','hc_y','tfs_deprecated','tfs_zulu_deprecated','fielder_2','umpire','sv_id','vx0','vy0','vz0','ax','ay','az','sz_top','sz_bot','hit_distance_sc','launch_speed','launch_angle','effective_speed','pitcher.1','fielder_2.1','fielder_3','fielder_4','fielder_5','fielder_6','fielder_7','fielder_8','fielder_9','release_pos_y','estimated_ba_using_speedangle','estimated_woba_using_speedangle','woba_value','woba_denom','babip_value','iso_value','launch_speed_angle','home_score','away_score','post_away_score','post_home_score','spin_axis','delta_home_win_exp','delta_run_exp','game_type','inning_topbot','des','player_name','post_bat_score','post_fld_score','if_fielding_alignment','of_fielding_alignment','release_extension','release_spin_rate','pitch_name','batter','events'])
df

['Slider' '4-Seam Fastball' 'Changeup' 'Curveball' 'Pitch Out' 'Sinker'
 nan 'Intentional Ball']


C:\Users\16086\AppData\Local\Temp\ipykernel_17784\3179309606.py:5: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('savant_data.csv')


,pitch_type,game_date,stand,type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,at_bat_number,pitch_number,bat_score,fld_score
0,SL,2023-04-28,L,X,1,2,NaN,NaN,519203.0,2,4,31,5,0,5
1,FF,2023-04-28,L,B,0,2,NaN,NaN,519203.0,2,4,31,4,0,5
2,SL,2023-04-28,L,S,0,2,NaN,NaN,519203.0,2,4,31,3,0,5
3,CH,2023-04-28,L,S,0,1,NaN,NaN,519203.0,2,4,31,2,0,5
4,SL,2023-04-28,L,S,0,0,NaN,NaN,519203.0,2,4,31,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18753,SL,2015-04-08,R,B,0,1,NaN,NaN,NaN,1,1,5,2,0,0
18754,FF,2015-04-08,R,S,0,0,NaN,NaN,NaN,1,1,5,1,0,0
18755,SI,2015-04-08,R,X,0,2,NaN,NaN,NaN,0,1,4,3,0,0
18756,FF,2015-04-08,R,S,0,1,NaN,NaN,NaN,0,1,4,2,0,0


In [177]:
df.pitch_type.value_counts()

FF    8438
SL    4754
CH    2413
SI    1721
CU    1305
IN       8
PO       2
Name: pitch_type, dtype: int64

In [178]:
df = df[pd.notnull(df['pitch_type'])]
df.isnull().sum()


pitch_type           0
game_date            0
stand                0
type                 0
balls                0
strikes              0
on_3b            17395
on_2b            16143
on_1b            14147
outs_when_up         0
inning               0
at_bat_number        0
pitch_number         0
bat_score            0
fld_score            0
dtype: int64

In [179]:
df = df[~df['pitch_type'].isin(['PO', 'IN'])]

In [180]:
from sklearn.preprocessing import LabelEncoder

le_pitch_type = LabelEncoder()
le_stand = LabelEncoder()
le_type = LabelEncoder()

df['pitch_type'] = le_pitch_type.fit_transform(df['pitch_type'])
df['stand'] = le_stand.fit_transform(df['stand'])
df['type'] = le_type.fit_transform(df['type'])


In [181]:
df.isnull().sum()

pitch_type           0
game_date            0
stand                0
type                 0
balls                0
strikes              0
on_3b            17385
on_2b            16141
on_1b            14139
outs_when_up         0
inning               0
at_bat_number        0
pitch_number         0
bat_score            0
fld_score            0
dtype: int64

In [182]:
df['on_3b'] = df['on_3b'].notna().astype(int)
df['on_2b'] = df['on_2b'].notna().astype(int)
df['on_1b'] = df['on_1b'].notna().astype(int)

# convert 'game_date' to datetime if it is not already
df['game_date'] = pd.to_datetime(df['game_date'])

# create 'years_since_2015' column
start_date = pd.Timestamp('2015-04-08')
df['years_since_2015'] = (df['game_date'] - start_date).dt.days / 365.25

# drop 'game_date' column as it is no longer needed
df = df.drop(columns='game_date')

df = df.fillna(0)

df = df[::-1]
df = df.reset_index(drop=True)
df

,pitch_type,stand,type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,at_bat_number,pitch_number,bat_score,fld_score,years_since_2015
0,2,1,1,0,0,0,0,0,0,1,4,1,0,0,0.000000
1,2,1,1,0,1,0,0,0,0,1,4,2,0,0,0.000000
2,3,1,2,0,2,0,0,0,0,1,4,3,0,0,0.000000
3,2,1,1,0,0,0,0,0,1,1,5,1,0,0,0.000000
4,4,1,0,0,1,0,0,0,1,1,5,2,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18626,4,0,1,0,0,0,0,1,2,4,31,1,0,5,8.054757
18627,0,0,1,0,1,0,0,1,2,4,31,2,0,5,8.054757
18628,4,0,1,0,2,0,0,1,2,4,31,3,0,5,8.054757
18629,2,0,0,0,2,0,0,1,2,4,31,4,0,5,8.054757


In [183]:
df.isnull().sum()

pitch_type          0
stand               0
type                0
balls               0
strikes             0
on_3b               0
on_2b               0
on_1b               0
outs_when_up        0
inning              0
at_bat_number       0
pitch_number        0
bat_score           0
fld_score           0
years_since_2015    0
dtype: int64

In [184]:
df.pitch_type.value_counts()

2    8438
4    4754
0    2413
3    1721
1    1305
Name: pitch_type, dtype: int64

In [185]:
df.pitch_type.value_counts()

2    8438
4    4754
0    2413
3    1721
1    1305
Name: pitch_type, dtype: int64

In [186]:
label_mapping = {label: index for index, label in enumerate(le_pitch_type.classes_)}
label_mapping

{'CH': 0, 'CU': 1, 'FF': 2, 'SI': 3, 'SL': 4}

In [187]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC

# Define the feature set (X) and the target (y)
X = df.drop(columns='pitch_type')
y = df['pitch_type']

smote_nc = SMOTENC(categorical_features=[0, 1, 2], random_state=42)
X_resampled, y_resampled = smote_nc.fit_resample(X, y)

# Split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [188]:
y.value_counts()

2    8438
4    4754
0    2413
3    1721
1    1305
Name: pitch_type, dtype: int64

In [189]:
y_resampled.value_counts()  

2    8438
3    8438
4    8438
1    8438
0    8438
Name: pitch_type, dtype: int64

In [190]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [191]:
from sklearn.metrics import classification_report

# Predict the pitch type for test data
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=le_pitch_type.classes_))



              precision    recall  f1-score   support

          CH       0.26      0.14      0.18       482
          CU       0.19      0.06      0.09       302
          FF       0.49      0.71      0.58      1679
          SI       0.31      0.16      0.21       351
          SL       0.37      0.31      0.34       913

    accuracy                           0.43      3727
   macro avg       0.32      0.28      0.28      3727
weighted avg       0.39      0.43      0.39      3727



In [192]:
from xgboost import XGBClassifier

# Create a model
xgb = XGBClassifier(random_state=42)

# Fit the model
xgb.fit(X_train, y_train)

# Predict the pitch type for test data
y_pred_xgb = xgb.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred_xgb, target_names=new_target_names))


              precision    recall  f1-score   support

          CH       0.31      0.17      0.22       482
          CU       0.28      0.08      0.12       302
          FF       0.51      0.73      0.60      1679
          SI       0.35      0.22      0.27       351
          SL       0.41      0.34      0.37       913

    accuracy                           0.46      3727
   macro avg       0.37      0.31      0.32      3727
weighted avg       0.43      0.46      0.43      3727



In [ ]:
from xgboost import XGBClassifier

# Create a model
xgb = XGBClassifier(random_state=42)

# Fit the model
xgb.fit(X_train, y_train)

# Predict the pitch type for test data
y_pred_xgb = xgb.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred_xgb, target_names=new_target_names))